# Transfer Indra data to (or delete from) FileDB with Globus
### (Assuming you have the necessary accounts and permissions)

https://globus-sdk-python.readthedocs.io/en/stable/tutorial/#step-1-get-a-client

http://globus.org

In [1]:
import globus_sdk
import indra_globus as ig
from IPython.display import HTML

In [2]:
CLIENT_ID = '94d2fffa-a76b-4e2d-b910-698cda67e60f' # This is for  me (Bridget); supply your own

In [3]:
client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow()
authorize_url = client.oauth2_get_authorize_url()

In [4]:
html_link = 'Go <a href="{}">here</a> to authenticate, and remember to login to globus and the jhuidies#dmztest collection'.format(authorize_url)
HTML(html_link)

In [5]:
# replace auth_code with value obtained from URL
auth_code='OyZzQfBs3w3dEzJGqdcaJNmY7CA9gv'
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

## Full simulation (skipping existing snaps)

In [ ]:
# For a full simulation, need the run number. If you only have the X_Y_Z, that's ok.
run_num = ig.get_run_num(5,0,5)

In [79]:
transfer = ig.SimTransfer(token_response,run_num)

In [12]:
transfer.populate_tasks()

snapdir_000 exists in /sciserver/filedb08-02/cosmo/indra/5_0_5/
snapdir_030 exists in /sciserver/filedb08-02/cosmo/indra/5_0_5/
snapdir_032 exists in /sciserver/filedb08-02/cosmo/indra/5_0_5/
snapdir_034 exists in /sciserver/filedb08-02/cosmo/indra/5_0_5/
snapdir_037 exists in /sciserver/filedb08-02/cosmo/indra/5_0_5/
snapdir_041 exists in /sciserver/filedb08-02/cosmo/indra/5_0_5/
snapdir_048 exists in /sciserver/filedb08-02/cosmo/indra/5_0_5/
snapdir_058 exists in /sciserver/filedb08-02/cosmo/indra/5_0_5/
snapdir_063 exists in /sciserver/filedb08-02/cosmo/indra/5_0_5/


In [13]:
transfer.sanity_checks()

55 snap_dir tasks, 14080 sub_id tasks, and 70 file tasks
Number of sub_id tasks should be 14080
First snap_dir task pair is ('/datascope/indra5/5_0_5/snapdir_001/', '/sciserver/filedb08-02/cosmo/indra/5_0_5/snapdir_001/')
First sub_id task pair is ('/datascope/indra5/5_0_5/postproc_001/sub_ids_001.0', '/sciserver/filedb08-02/cosmo/indra/5_0_5/postproc_001/sub_ids_001.0')
First file task pair is ('/datascope/indra5/5_0_5/PIDs.txt', '/sciserver/filedb08-02/cosmo/indra/5_0_5/PIDs.txt')


In [14]:
tdata = transfer.submit_transfer() # for testing purposes, don't actually submit transfer, but get the tdata
# To transfer for real:
#transfer.submit_transfer(for_real=True)

Transfer submitted (but not really, just testing code!) Returning tdata.


In [15]:
tdata.keys()

dict_keys(['DATA_TYPE', 'submission_id', 'source_endpoint', 'destination_endpoint', 'verify_checksum', 'preserve_timestamp', 'encrypt_data', 'recursive_symlinks', 'label', 'sync_level', 'DATA'])

In [17]:
tdata['label']

'Indra_5_0_5'

### Delete full simulation, keeping priority snapshots (determined by snaps in skip_example, defaults to 2_0_1)

In [81]:
snapdata,filedata = transfer.submit_delete()
# OR
# transfer.submit_delete(for_real=True)

Keeping snapdir_000 in /sciserver/filedb08-02/cosmo/indra/5_0_5/
Keeping snapdir_030 in /sciserver/filedb08-02/cosmo/indra/5_0_5/
Keeping snapdir_032 in /sciserver/filedb08-02/cosmo/indra/5_0_5/
Keeping snapdir_034 in /sciserver/filedb08-02/cosmo/indra/5_0_5/
Keeping snapdir_037 in /sciserver/filedb08-02/cosmo/indra/5_0_5/
Keeping snapdir_041 in /sciserver/filedb08-02/cosmo/indra/5_0_5/
Keeping snapdir_048 in /sciserver/filedb08-02/cosmo/indra/5_0_5/
Keeping snapdir_058 in /sciserver/filedb08-02/cosmo/indra/5_0_5/
Keeping snapdir_063 in /sciserver/filedb08-02/cosmo/indra/5_0_5/
Not deleting; returning snapdata, filedata
To delete, call NAME.tc.submit_delete(snapdata) and (filedata) or set for_real=True


In [83]:
snapdata['DATA'][0]

{'DATA_TYPE': 'delete_item',
 'path': '/sciserver/filedb08-02/cosmo/indra/5_0_5/snapdir_001/'}

In [84]:
filedata['DATA'][0]

{'DATA_TYPE': 'delete_item',
 'path': '/sciserver/filedb08-02/cosmo/indra/5_0_5/postproc_001/sub_ids_001.0'}

In [85]:
snapdata.keys()

dict_keys(['DATA_TYPE', 'submission_id', 'endpoint', 'recursive', 'label', 'DATA'])

In [86]:
print(snapdata['recursive'],filedata['recursive'])

True False


In [87]:
filedata['DATA'][-1]

{'DATA_TYPE': 'delete_item',
 'path': '/sciserver/filedb08-02/cosmo/indra/5_0_5/postproc_062/sub_ids_062.255'}

## Set of snapshots

In [ ]:
snapnum = 42

In [43]:
# 2_0_0 to 7_7_7
snaptransfer = ig.SnapTransfer(token_response,snapnum,include_7=True)
print(snaptransfer.dlocs[0],snaptransfer.flocs[0])

Setting up transfer of 384 runs, starting with 128, of snapnum 42
/datascope/indra2/2_0_0/ /sciserver/filedb02-03/cosmo/indra/2_0_0/


In [44]:
# 0_0_0 to 1_7_7
snaptransfer= ig.SnapTransfer(token_response,snapnum,runfirst=0,nruns=128)
print(snaptransfer.dlocs[0],snaptransfer.flocs[0])

Setting up transfer of 128 runs, starting with 0, of snapnum 42
/datascope/indra0/0_0_0/ /sciserver/filedb08-01/cosmo/indra/0_0_0/


In [99]:
# 2_0_0 to 6_7_7 (default)
snaptransfer = ig.SnapTransfer(token_response,snapnum)
print(snaptransfer.dlocs[0],snaptransfer.flocs[0])

Setting up transfer of 320 runs, starting with 128, of snapnum 42
/datascope/indra2/2_0_0/ /sciserver/filedb02-03/cosmo/indra/2_0_0/


In [55]:
snaptransfer.populate_tasks()

snapdir_042 exists in /sciserver/filedb02-03/cosmo/indra/2_0_0/
snapdir_042 exists in /sciserver/filedb12-01/cosmo/indra/3_0_0/
snapdir_042 exists in /sciserver/filedb09-02/cosmo/indra/4_0_0/
snapdir_042 exists in /sciserver/filedb06-03/cosmo/indra/5_0_0/
snapdir_042 exists in /sciserver/filedb04-01/cosmo/indra/6_0_0/


In [56]:
snaptransfer.sanity_checks()

315 snap_dir tasks and 80640 sub_id tasks
Number of sub_id tasks should be 80640
First snap_dir task pair is ('/datascope/indra2/2_0_1/snapdir_042/', '/sciserver/filedb03-01/cosmo/indra/2_0_1/snapdir_042/')
First sub_id task pair is ('/datascope/indra2/2_0_1/postproc_042/sub_ids_042.0', '/sciserver/filedb03-01/cosmo/indra/2_0_1/postproc_042/sub_ids_042.0')
Last snap_dir task pair is ('/datascope/indra6/6_7_7/snapdir_042/', '/sciserver/filedb01-01/cosmo/indra/6_7_7/snapdir_042/')
Last sub_id task pair is ('/datascope/indra6/6_7_7/postproc_042/sub_ids_042.255', '/sciserver/filedb01-01/cosmo/indra/6_7_7/postproc_042/sub_ids_042.255')


In [ ]:
tdata = snaptransfer.submit_transfer()
#snaptransfer.submit_transfer(for_real = True)

### Delete set of snapshots, keeping those in full simulations (skip_list defaults to current set)

In [100]:
snapdata,filedata = snaptransfer.submit_delete()

Keeping snapdir_042 in /sciserver/filedb02-03/cosmo/indra/2_0_0/
Keeping snapdir_042 in /sciserver/filedb12-01/cosmo/indra/3_0_0/
Keeping snapdir_042 in /sciserver/filedb09-02/cosmo/indra/4_0_0/
Keeping snapdir_042 in /sciserver/filedb06-03/cosmo/indra/5_0_0/
Keeping snapdir_042 in /sciserver/filedb04-01/cosmo/indra/6_0_0/
Not deleting; returning snapdata, filedata
To delete, call NAME.tc.submit_delete(snapdata) and (filedata) or set for_real=True


In [101]:
snapdata['DATA'][0]

{'DATA_TYPE': 'delete_item',
 'path': '/sciserver/filedb03-01/cosmo/indra/2_0_1/snapdir_042/'}

In [102]:
filedata['DATA'][0]

{'DATA_TYPE': 'delete_item',
 'path': '/sciserver/filedb03-01/cosmo/indra/2_0_1/postproc_042/sub_ids_042.0'}

In [103]:
print(snapdata['recursive'],filedata['recursive'])

True False


In [104]:
filedata['DATA'][-1]

{'DATA_TYPE': 'delete_item',
 'path': '/sciserver/filedb01-01/cosmo/indra/6_7_7/postproc_042/sub_ids_042.255'}

## Series of 64 runs: FFT_DATA directories and sub_tab files
IF, for example, you are adding 7 series data, do this step before transferring full sims or sets of snapshots.

In [105]:
transfer = ig.SeriesTransfer(token_response,series_num = 7)

In [59]:
transfer.flocs[0]

'/sciserver/filedb01-02/cosmo/indra/7_0_0/'

In [73]:
transfer.make_dirs() # For these, directories need to exist

7_0_0  exists in  /sciserver/filedb01-02/cosmo/indra/
7_0_1  exists in  /sciserver/filedb01-03/cosmo/indra/
7_0_2  exists in  /sciserver/filedb02-01/cosmo/indra/
7_0_3  exists in  /sciserver/filedb02-02/cosmo/indra/
7_0_4  exists in  /sciserver/filedb02-03/cosmo/indra/
7_0_5  exists in  /sciserver/filedb03-01/cosmo/indra/
7_0_6  exists in  /sciserver/filedb03-02/cosmo/indra/
7_0_7  exists in  /sciserver/filedb03-03/cosmo/indra/
7_1_0  exists in  /sciserver/filedb04-01/cosmo/indra/
7_1_1  exists in  /sciserver/filedb04-02/cosmo/indra/
7_1_2  exists in  /sciserver/filedb04-03/cosmo/indra/
7_1_3  exists in  /sciserver/filedb05-01/cosmo/indra/
7_1_4  exists in  /sciserver/filedb05-02/cosmo/indra/
7_1_5  exists in  /sciserver/filedb05-03/cosmo/indra/
7_1_6  exists in  /sciserver/filedb06-01/cosmo/indra/
7_1_7  exists in  /sciserver/filedb06-02/cosmo/indra/
7_2_0  exists in  /sciserver/filedb06-03/cosmo/indra/
7_2_1  exists in  /sciserver/filedb07-01/cosmo/indra/
7_2_2  exists in  /sciserver

In [74]:
transfer.populate_tasks()

Populating sub_tab task pairs (could take a while)


In [75]:
transfer.sanity_checks()

64 fft tasks and 1048576 sub_tab tasks
Number of sub_tab tasks should be 1048576
First fft task pair is ('/datascope/indra7/7_0_0/FFT_DATA/', '/sciserver/filedb01-02/cosmo/indra/7_0_0/FFT_DATA/')
First sub_tab task pair is ('/datascope/indra7/7_0_0/postproc_000/sub_tab_000.0', '/sciserver/filedb01-02/cosmo/indra/7_0_0/postproc_000/sub_tab_000.0')
Last fft task pair is ('/datascope/indra7/7_7_7/FFT_DATA/', '/sciserver/filedb10-02/cosmo/indra/7_7_7/FFT_DATA/')
Last sub_tab task pair is ('/datascope/indra7/7_7_7/postproc_063/sub_tab_063.255', '/sciserver/filedb10-02/cosmo/indra/7_7_7/postproc_063/sub_tab_063.255')


In [76]:
tdata = transfer.submit_transfer()

Transfer Indra_7 submitted (but not really, just testing code!) Returning tdata.


In [77]:
tdata['label']

'Indra_7'

### Delete EVERYTHING in a full 64-run series, including top-level folders
(But please don't!)

In [106]:
ddata = transfer.submit_delete()

Not deleting; returning data
To delete, call NAME.tc.submit_delete(data) or set for_real=True


In [107]:
ddata['label']

'Indra_delete_7_series'

In [108]:
ddata['DATA'][0]

{'DATA_TYPE': 'delete_item',
 'path': '/sciserver/filedb01-02/cosmo/indra/7_0_0'}

In [109]:
len(ddata['DATA'])

64

In [110]:
# I created 7 series folders to test, so now I can test deleting them (they are empty anyway)
transfer.submit_delete(for_real=True)

Deleting!


#### SUCCESS!